In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping



In [2]:
data=pd.read_csv('Churn_Modelling.csv')

In [3]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [4]:
label_encoder_gender = LabelEncoder()

In [5]:
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])


In [6]:
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')

In [7]:
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()


In [8]:
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [9]:
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [10]:
X= data.drop('Exited', axis=1)

In [11]:
y = data['Exited']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [13]:
scaler = StandardScaler()

In [14]:
X_train = scaler.fit_transform(X_train)

In [15]:
X_test = scaler.transform(X_test)

In [16]:
import pickle

In [17]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender, file)

In [18]:
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [19]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler,file)

In [30]:
def create_model(neurons=32, layers=1):
    model=Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))
    
    for _ in range(layers-1):
        model.add(Dense(neurons, activation='relu'))
        
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])
    
    return model 

In [31]:
model = KerasClassifier(layers=1, neurons=32,build_fn=create_model,verbose=1)

In [32]:
param_grid = {
    'neurons':[16,32,64,128],
    'layers':[1,2],
    'epochs':[50,100]
}

In [36]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1,cv=3, verbose=1)

In [37]:
grid_result=grid.fit(X_train,y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


d:\ADITI\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
d:\ADITI\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6955 - loss: 0.5823
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7959 - loss: 0.4581
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8171 - loss: 0.4211
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8263 - loss: 0.4095
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8380 - loss: 0.3981
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8432 - loss: 0.3850
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8521 - loss: 0.3648
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8592 - loss: 0.3572
Epoch 9/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8545 - loss: 0.3533
Epoch 10/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8569 - loss: 0.3514
Epoch 11/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8532 - loss: 0.3505
Epoch 12/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

In [38]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.856624 using {'epochs': 50, 'layers': 1, 'neurons': 16}
